## XGBoost-Optunaのサンプルコード

In [1]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# XGBoost
import xgboost as xgb

# Optuna
import optuna
from sklearn.model_selection import cross_val_score

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns = boston.feature_names)
# 目的変数の追加
df['MEDV'] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [3]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 0:df.shape[1]-1],
                                                    df.iloc[:, df.shape[1]-1],
                                                    test_size=TEST_SIZE,
                                                    random_state=RANDOM_STATE)

In [4]:
def objective(trial):
    
    eta =  trial.suggest_loguniform('eta', 1e-8, 1.0)
    gamma = trial.suggest_loguniform('gamma', 1e-8, 1.0)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_child_weight = trial.suggest_loguniform('min_child_weight', 1e-8, 1.0)
    subsample = trial.suggest_uniform('subsample', 0.0, 1.0)
    reg_lambda = trial.suggest_uniform('reg_lambda', 0.0, 1000.0)
    reg_alpha = trial.suggest_uniform('reg_alpha', 0.0, 1000.0)


    model =xgb.XGBRegressor(eta = eta,
                            gamma = gamma,
                            max_depth = max_depth,
                            min_child_weight = min_child_weight,
                            subsample = subsample,
                            reg_lambda = reg_lambda,
                            reg_alpha = reg_alpha
                           )

    score = cross_val_score(model,
                            x_train,
                            y_train,
                            cv=5,
                            scoring="neg_mean_absolute_error"
                           )
    print(score)
    mae = score.mean()

    return mae

In [5]:
%%time
# optunaで最適値を見つける
# 注：cross_val_scoreの出力は全て高いほど良い
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

[I 2021-05-08 10:01:04,954] A new study created in memory with name: no-name-37756620-6678-4d42-bb5e-cf43c3687513
[I 2021-05-08 10:01:05,158] Trial 0 finished with value: -21.32003762225163 and parameters: {'eta': 4.893787827381806e-05, 'gamma': 1.6790883203191978e-05, 'max_depth': 3, 'min_child_weight': 0.09852275197108669, 'subsample': 0.7277910086840357, 'reg_lambda': 666.3837025757402, 'reg_alpha': 613.0101247081783}. Best is trial 0 with value: -21.32003762225163.
[I 2021-05-08 10:01:05,317] Trial 1 finished with value: -21.297835488119244 and parameters: {'eta': 0.00018102320991114995, 'gamma': 4.627784993811587e-05, 'max_depth': 6, 'min_child_weight': 0.0026322297907259763, 'subsample': 0.49940689780553293, 'reg_lambda': 818.2063393917749, 'reg_alpha': 956.8001163820826}. Best is trial 1 with value: -21.297835488119244.


[-20.23018981 -22.19254566 -20.73526374 -21.30085522 -22.14133368]
[-20.20757366 -22.1707672  -20.71289233 -21.2786817  -22.11926255]


[I 2021-05-08 10:01:05,482] Trial 2 finished with value: -21.05611203412068 and parameters: {'eta': 0.0009950512747066151, 'gamma': 0.00027484208928785713, 'max_depth': 14, 'min_child_weight': 0.00035029765320008133, 'subsample': 0.4123791328365103, 'reg_lambda': 841.8183338736751, 'reg_alpha': 11.390419213288316}. Best is trial 2 with value: -21.05611203412068.
[I 2021-05-08 10:01:05,626] Trial 3 finished with value: -21.293891472616316 and parameters: {'eta': 5.8305046710018006e-05, 'gamma': 0.1825276374681468, 'max_depth': 10, 'min_child_weight': 6.533711212703294e-08, 'subsample': 0.9386640828993914, 'reg_lambda': 365.32049477547713, 'reg_alpha': 733.134571768201}. Best is trial 2 with value: -21.05611203412068.


[-19.96324463 -21.93155671 -20.47008709 -21.03681976 -21.87885199]
[-20.20367254 -22.1666999  -20.70893929 -21.27471978 -22.11542586]


[I 2021-05-08 10:01:05,794] Trial 4 finished with value: -21.34301048020375 and parameters: {'eta': 2.921317139379561e-06, 'gamma': 0.10582283937976816, 'max_depth': 3, 'min_child_weight': 0.32415863481286483, 'subsample': 0.525106661052768, 'reg_lambda': 908.2615300723569, 'reg_alpha': 477.4042265543027}. Best is trial 2 with value: -21.05611203412068.
[I 2021-05-08 10:01:05,938] Trial 5 finished with value: -20.646539015569807 and parameters: {'eta': 0.001215779782909126, 'gamma': 3.7078809850721246e-08, 'max_depth': 13, 'min_child_weight': 0.00856745351520768, 'subsample': 0.44052827445019493, 'reg_lambda': 372.30913962138976, 'reg_alpha': 39.57308691999329}. Best is trial 5 with value: -20.646539015569807.


...

[-2.10667663 -2.41663367 -2.42781584 -2.63514261 -3.07929174]


[I 2021-05-08 10:06:17,437] Trial 498 finished with value: -2.8537146895255576 and parameters: {'eta': 0.2853908893650242, 'gamma': 2.0195975839541739e-07, 'max_depth': 14, 'min_child_weight': 1.0035268483648305e-08, 'subsample': 0.48449998503003194, 'reg_lambda': 54.33898677520666, 'reg_alpha': 83.22931678336641}. Best is trial 368 with value: -2.096921031392651.


[-2.33705345 -2.53567934 -2.80439286 -2.85680035 -3.73464745]


[I 2021-05-08 10:06:18,167] Trial 499 finished with value: -2.4202989725301296 and parameters: {'eta': 0.14283484373447333, 'gamma': 1.117900530061839e-07, 'max_depth': 14, 'min_child_weight': 1.0085702908491227e-08, 'subsample': 0.5225486120225962, 'reg_lambda': 16.79469672450902, 'reg_alpha': 16.968933738947793}. Best is trial 368 with value: -2.096921031392651.


[-1.92703959 -2.29776026 -2.33649292 -2.51622667 -3.02397542]
CPU times: user 9min 35s, sys: 16 s, total: 9min 51s
Wall time: 5min 13s


In [6]:
# チューニングしたハイパーパラメーターをフィット
optimised_model = xgb.XGBRegressor(eta = study.best_params['eta'],
                                   gamma = study.best_params['gamma'],
                                   max_depth = study.best_params['max_depth'],
                                   min_child_weight = study.best_params['min_child_weight'],
                                   subsample = study.best_params['subsample'],
                                   reg_lambda = study.best_params['reg_lambda'],
                                   reg_alpha = study.best_params['reg_alpha']
                                  )

optimised_model.fit(x_train ,y_train)

# XGBoost推論
y_pred = optimised_model.predict(x_test)

In [7]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame({'R2': r2_score(true, pred),
                          'MAE': mean_absolute_error(true, pred),
                          'MSE': mean_squared_error(true, pred),
                          'RMSE': np.sqrt(mean_squared_error(true, pred))},
                           index = ['scores'])
    return scores

In [8]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE        MSE      RMSE
scores  0.879891  2.475511  12.561133  3.544169
